In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import pandas as pd

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/My Drive/Zevi')
os.listdir()

['Zevi_search_dataset.json',
 'd2vModel.d2v',
 'new_data.csv',
 'dict_file',
 'lsi_model',
 'tfidf_model',
 'lsi_model.projection',
 'tfidf_model_mm.index',
 'tfidf_model_mm',
 'lsi_model_mm',
 'lsi_model_mm.index',
 'matrix_index',
 'Final Model.ipynb',
 'Zevi model.ipynb',
 'Classifier Model.ipynb']

# Enter Data

In [ ]:
data = pd.read_json('Zevi_search_dataset.json')
data.head()

,name,description,brand,categories,hierarchicalCategories,type,price,price_range,image,url,free_shipping,rating,popularity,objectID
0,Amazon - Fire TV Stick with Alexa Voice Remote...,"Enjoy smart access to videos, games and apps w...",Amazon,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,39.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5477...,https://api.bestbuy.com/click/-/5477500/pdp,False,4,21469,5477500
1,Google - Chromecast - Black,Google Chromecast: Enjoy a world of entertainm...,Google,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,35.00,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/4397...,https://api.bestbuy.com/click/-/4397400/pdp,False,4,21468,4397400
2,"Dell - Inspiron 15.6"" Touch-Screen Laptop - In...",Dell Inspiron Laptop: Get speed and performanc...,Dell,"[Computers & Tablets, Laptops]","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Burst skus,499.99,200 - 500,https://cdn-demo.algolia.com/bestbuy-0118/5588...,https://api.bestbuy.com/click/-/5588602/pdp,True,4,21467,5588602
3,Amazon - Echo Dot,Deliver your favorite playlist anywhere in you...,Amazon,[Smart Home],{'lvl0': 'Smart Home'},Voice assistants,49.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5578...,https://api.bestbuy.com/click/-/5578851/pdp,True,4,21466,5578851
4,"Apple - MacBook Air® (Latest Model) - 13.3"" Di...","MacBook Air features up to 8GB of memory, a fi...",Apple,"[Computers & Tablets, Laptops, All Laptops, Ma...","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Apple,999.99,500 - 2000,https://cdn-demo.algolia.com/bestbuy-0118/6443...,https://api.bestbuy.com/click/-/6443034/pdp,True,4,21465,6443034


In [ ]:
print(len(data))

21469


# Preprocessing

In [ ]:
pip install spacy

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import string
import re

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [ ]:
print ('Cleaning and Tokenizing...')


%time data['description_tokenized'] = data['description'].map(lambda x: spacy_tokenizer(x))

Cleaning and Tokenizing...
CPU times: user 22min 49s, sys: 30.6 s, total: 23min 20s
Wall time: 24min 10s


In [ ]:
data.head()

,name,description,brand,categories,hierarchicalCategories,type,price,price_range,image,url,free_shipping,rating,popularity,objectID,description_tokenized
0,Amazon - Fire TV Stick with Alexa Voice Remote...,"Enjoy smart access to videos, games and apps w...",Amazon,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,39.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5477...,https://api.bestbuy.com/click/-/5477500/pdp,False,4,21469,5477500,"[enjoy, smart, access, video, game, app, amazo..."
1,Google - Chromecast - Black,Google Chromecast: Enjoy a world of entertainm...,Google,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,35.00,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/4397...,https://api.bestbuy.com/click/-/4397400/pdp,False,4,21468,4397400,"[google, chromecast, enjoy, world, entertainme..."
2,"Dell - Inspiron 15.6"" Touch-Screen Laptop - In...",Dell Inspiron Laptop: Get speed and performanc...,Dell,"[Computers & Tablets, Laptops]","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Burst skus,499.99,200 - 500,https://cdn-demo.algolia.com/bestbuy-0118/5588...,https://api.bestbuy.com/click/-/5588602/pdp,True,4,21467,5588602,"[dell, inspiron, laptop, speed, performance, i..."
3,Amazon - Echo Dot,Deliver your favorite playlist anywhere in you...,Amazon,[Smart Home],{'lvl0': 'Smart Home'},Voice assistants,49.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5578...,https://api.bestbuy.com/click/-/5578851/pdp,True,4,21466,5578851,"[deliver, favorite, playlist, home, amazon, ec..."
4,"Apple - MacBook Air® (Latest Model) - 13.3"" Di...","MacBook Air features up to 8GB of memory, a fi...",Apple,"[Computers & Tablets, Laptops, All Laptops, Ma...","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Apple,999.99,500 - 2000,https://cdn-demo.algolia.com/bestbuy-0118/6443...,https://api.bestbuy.com/click/-/6443034/pdp,True,4,21465,6443034,"[macbook, air, feature, memory, fifth, generat..."


In [ ]:
data.to_json('new_data1.json')

In [ ]:
descriptions = data['description_tokenized']
descriptions[0:5]

0    [enjoy, smart, access, video, game, app, amazo...
1    [google, chromecast, enjoy, world, entertainme...
2    [dell, inspiron, laptop, speed, performance, i...
3    [deliver, favorite, playlist, home, amazon, ec...
4    [macbook, air, feature, memory, fifth, generat...
Name: description_tokenized, dtype: object

# Creating Dictionary and Corpus

In [ ]:
import numpy as np 
import gensim
import operator

In [ ]:
from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(descriptions)

#filter out terms which occur in more than 50% of the descriptions
dictionary.filter_extremes(no_above=0.5)

CPU times: user 719 ms, sys: 903 µs, total: 720 ms
Wall time: 724 ms


In [ ]:
dictionary.save('dict_file', separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)

In [ ]:
dictionary = corpora.Dictionary.load('dict_file', mmap=None)

In [ ]:
corpus = [dictionary.doc2bow(description) for description in descriptions]

# Fitting Data in LSI Model

In [ ]:
%time tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=dictionary)

CPU times: user 126 ms, sys: 0 ns, total: 126 ms
Wall time: 129 ms
CPU times: user 9.9 s, sys: 1.57 s, total: 11.5 s
Wall time: 7.9 s


In [ ]:
tfidf_model.save('tfidf_model', separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
tfidf_model = gensim.models.TfidfModel.load('tfidf_model', mmap=None)
lsi_model.save('lsi_model',separately=None, sep_limit=10485760, pickle_protocol=2)
lsi_model = gensim.models.LsiModel.load('lsi_model',mmap=None)

In [ ]:
%time gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('lsi_model_mm',lsi_model[tfidf_model[corpus]])

CPU times: user 4.05 s, sys: 74.6 ms, total: 4.13 s
Wall time: 5.31 s
CPU times: user 15.9 s, sys: 205 ms, total: 16.1 s
Wall time: 18.4 s


In [ ]:
#Load the indexed corpus
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

print(tfidf_corpus)
print(lsi_corpus)

MmCorpus(21469 documents, 5755 features, 471801 non-zero entries)
MmCorpus(21469 documents, 200 features, 4293800 non-zero entries)


In [ ]:
from gensim.similarities import MatrixSimilarity

%time index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

CPU times: user 5.36 s, sys: 146 ms, total: 5.51 s
Wall time: 5.64 s


In [ ]:
index.save('matrix_index', separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
index = MatrixSimilarity.load('matrix_index', mmap=None)

# Predictions

In [ ]:
from operator import itemgetter

def search_similar_products(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    index.num_best = 5

    product_list = index[query_lsi]

    product_list.sort(key=itemgetter(1), reverse=True)
    names = []

    for j, product in enumerate(product_list):

        names.append (
            {
                'Name': data['name'][product[0]],
            }

        )
        if j == (index.num_best-1):
            break

    return pd.DataFrame(names, columns=['Name'])

In [ ]:
search_similar_products('dell inspiron')

,Name
0,"Dell - Inspiron 15.6"" Laptop - Intel Core i3 -..."
1,"Dell - Inspiron 3459 23.8"" Touch-Screen All-In..."
2,"Dell - Inspiron 15.6"" Laptop - Intel Core i7 -..."
3,"Dell - Inspiron 2-in-1 13.3"" Touch-Screen Lapt..."
4,"Dell - Inspiron 19.5"" Touch-Screen All-In-One ..."


In [ ]:
dist_price_ranges = set([])
for r in data['price_range']:
  if(r not in dist_price_ranges):
    dist_price_ranges.add(r)
print(dist_price_ranges)

{'500 - 2000', '50 - 100', '1 - 50', '100 - 200', '200 - 500', '> 2000'}


In [ ]:
dist_categories = set([])
for r in data['categories']:
  if(len(r)==0):
    continue
  if(r[0] not in dist_categories):
    dist_categories.add(r[0])
print(len(dist_categories))

39
